In [1]:
from PIL import Image
import numpy as np
import h5py

In [2]:
h5py.enable_ipython_completer()

In [3]:
import keras
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

Using TensorFlow backend.


In [4]:
import os

In [12]:
def get_resized_image(image_dir, dimensions):
    img = Image.open(image_dir)
    return img.resize(dimensions)

In [5]:
def resize_directory_images(in_directory, out_directory):
    for subdir, dirs, files in os.walk(in_directory):
        for file in files:
            if file != '.DS_Store':
                new_img = get_resized_image(in_directory + file, (128,128))
                new_img.save(out_directory + file)

In [47]:
#resize_directory_images('./training_data/hotdogs/', './resized_data/hotdogs/')
#resize_directory_images('./training_data/hotdogs_testing/', './resized_data/hotdogs_testing/')
#resize_directory_images('./training_data/not hotdogs/', './resized_data/not hotdogs/')
#resize_directory_images('./training_data/notdogs_testing/', './resized_data/notdogs_testing/')
# resize_directory_images('./more/', './resized_data/train/hotdogs/')

In [5]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

In [159]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(128, 128, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [160]:
batch_size = 16

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.9,
        zoom_range=0.9,    
        rotation_range=5,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255,)

In [161]:
train_generator = train_datagen.flow_from_directory(
        './resized_data/train/',  # this is the target directory
        target_size=(128, 128),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels


validation_generator = test_datagen.flow_from_directory(
        './resized_data/test/',
        target_size=(128, 128),
        batch_size=batch_size,
        class_mode='binary')

Found 1185 images belonging to 2 classes.
Found 296 images belonging to 2 classes.


In [162]:
model.fit_generator(
        train_generator,
        steps_per_epoch=1200 // batch_size,
        epochs=30,
        validation_data=validation_generator,
        validation_steps=300 // batch_size)

Epoch 1/30
75/75 [==============================] - 23s - loss: 0.7497 - acc: 0.5416 - val_loss: 0.6903 - val_acc: 0.5312
Epoch 2/30
75/75 [==============================] - 22s - loss: 0.6667 - acc: 0.5758 - val_loss: 0.7438 - val_acc: 0.5107
Epoch 3/30
75/75 [==============================] - 22s - loss: 0.6574 - acc: 0.6191 - val_loss: 0.6796 - val_acc: 0.5393
Epoch 4/30
75/75 [==============================] - 22s - loss: 0.6389 - acc: 0.6383 - val_loss: 0.8130 - val_acc: 0.5571
Epoch 5/30
75/75 [==============================] - 23s - loss: 0.6303 - acc: 0.6341 - val_loss: 0.7101 - val_acc: 0.5250
Epoch 6/30
75/75 [==============================] - 22s - loss: 0.5945 - acc: 0.6716 - val_loss: 0.6212 - val_acc: 0.7071
Epoch 7/30
75/75 [==============================] - 22s - loss: 0.5878 - acc: 0.6841 - val_loss: 0.7345 - val_acc: 0.5286
Epoch 8/30
75/75 [==============================] - 22s - loss: 0.5892 - acc: 0.6841 - val_loss: 0.6708 - val_acc: 0.5429
Epoch 9/30
75/75 [======

In [104]:
hotdog_img = get_resized_image('./resized_data/train/hotdogs/20.jpg', (128,128))
#img = get_resized_image('./resized_data/train/notdogs/30.jpg', (128, 128))
x = img_to_array(hotdog_img)
x_expanded = np.expand_dims(x, axis=0)
y = model.predict(x_expanded, batch_size=1, verbose=1)
print(y)

1/1 [==============================] - 0s
[[ 1.]]


In [105]:
notdog_img = get_resized_image('./resized_data/train/notdogs/1.jpg', (128, 128))
x = img_to_array(notdog_img)
x_expanded = np.expand_dims(x, axis=0)
y = model.predict(x_expanded, batch_size=1)
print(y)

[[ 1.]]


In [95]:
model.load_weights('hotdog.h5')

In [69]:
y

array([[ 1.]], dtype=float32)

In [134]:
hotdog_predictions = []
for subdir, dirs, files in os.walk('./resized_data/test/hotdogs/'):
        for file in files:
            if file != '.DS_Store':
                img = Image.open('./resized_data/test/hotdogs/' + file)
                x = img_to_array(img)
                x = np.expand_dims(x, axis=0)
                hotdog_predictions.append(model.predict(x))

hotdog_predictions

[array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 0.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 0.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 0.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 0.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([

In [132]:
nothotdog_predictions = []
for subdir, dirs, files in os.walk('./resized_data/test/notdogs/'):
        for file in files:
            if file != '.DS_Store':
                img = Image.open('./resized_data/test/notdogs/' + file)
                x = img_to_array(img)
                x = np.expand_dims(x, axis=0)
                nothotdog_predictions.append(model.predict(x))

nothotdog_predictions

[array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 0.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([[ 1.]], dtype=float32),
 array([